In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)
matplotlib 2.1.2
numpy 1.19.1
pandas 0.22.0
sklearn 0.19.1
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


In [3]:
source_dir = "./generate_csv/"

def get_filenames_by_prefix(source_dir, prefix_name):
    all_files = os.listdir(source_dir)
    results = []
    for filename in all_files:
        if filename.startswith(prefix_name):
            results.append(os.path.join(source_dir, filename))
    return results

train_filenames = get_filenames_by_prefix(source_dir, "train")
valid_filenames = get_filenames_by_prefix(source_dir, "valid")
test_filenames = get_filenames_by_prefix(source_dir, "test")

import pprint
pprint.pprint(train_filenames)
pprint.pprint(valid_filenames)
pprint.pprint(test_filenames)

['./generate_csv/train_12.csv',
 './generate_csv/train_18.csv',
 './generate_csv/train_19.csv',
 './generate_csv/train_13.csv',
 './generate_csv/train_07.csv',
 './generate_csv/train_11.csv',
 './generate_csv/train_10.csv',
 './generate_csv/train_08.csv',
 './generate_csv/train_14.csv',
 './generate_csv/train_04.csv',
 './generate_csv/train_17.csv',
 './generate_csv/train_02.csv',
 './generate_csv/train_09.csv',
 './generate_csv/train_01.csv',
 './generate_csv/train_03.csv',
 './generate_csv/train_00.csv',
 './generate_csv/train_06.csv',
 './generate_csv/train_05.csv',
 './generate_csv/train_16.csv',
 './generate_csv/train_15.csv']
['./generate_csv/valid_09.csv',
 './generate_csv/valid_06.csv',
 './generate_csv/valid_07.csv',
 './generate_csv/valid_03.csv',
 './generate_csv/valid_05.csv',
 './generate_csv/valid_00.csv',
 './generate_csv/valid_02.csv',
 './generate_csv/valid_01.csv',
 './generate_csv/valid_08.csv',
 './generate_csv/valid_04.csv']
['./generate_csv/test_08.csv',
 './gener

In [4]:
def parse_csv_line(line, n_fields=9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x, y

def csv_reader_dataset(filenames, n_readers=5, batch_size=32, n_parse_threads=5, shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length=n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_csv_line, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32
train_set = csv_reader_dataset(train_filenames, batch_size=batch_size)
valid_set = csv_reader_dataset(valid_filenames, batch_size=batch_size)
test_set = csv_reader_dataset(test_filenames, batch_size=batch_size)

<ShuffleDataset shapes: (), types: tf.string>
<ShuffleDataset shapes: (), types: tf.string>
<ShuffleDataset shapes: (), types: tf.string>


In [8]:
# 数据格式装换,转换待训练的数据格式
def serialize_example(x, y):
    """Convert x, y to tf.train.Example and serialize"""
    input_features = tf.train.FloatList(value=x)
    label = tf.train.FloatList(value=y)
    features = tf.train.Features(
        feature = {
            "input_features": tf.train.Feature(float_list = input_features),
            "label": tf.train.Feature(float_list = label)
        }
    )
    example = tf.train.Example(features=features)
    return example.SerializeToString()

def csv_dataset_to_tfrecords(base_filename, dataset, n_shards, steps_per_shard, compression_type=None):
    """
    n_shards: 保存的文件的个数
    steps_per_shard: 每一个小文件在dataset上走多少步
    """
    options = tf.io.TFRecordOptions(
        compression_type=compression_type
    )
    all_filenames = []
    for shard_id in range(n_shards):
        filename_fullpath = '{}_{:05d}-of-{:05d}'.format(base_filename, shard_id, n_shards)
        with tf.io.TFRecordWriter(filename_fullpath, options) as writer:
            for x_batch, y_batch in dataset.take(steps_per_shard):
                for x_example, y_example in zip(x_batch, y_batch):
                    writer.write(serialize_example(x_example, y_example))
        all_filenames.append(filename_fullpath)
    return all_filenames

In [9]:
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = "generate_tfrecords"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(train_basename, train_set, n_shards, train_steps_per_shard, None)
valid_tfrecord_filenames = csv_dataset_to_tfrecords(valid_basename, valid_set, n_shards, valid_steps_per_shard, None)
test_tfrecords_filenames = csv_dataset_to_tfrecords(test_basename, test_set, n_shards, test_steps_per_shard, None)

In [10]:
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = "generate_tfrecords_zip"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard,
    compression_type = "GZIP")
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, n_shards, valid_steps_per_shard,
    compression_type = "GZIP")
test_tfrecord_fielnames = csv_dataset_to_tfrecords(
    test_basename, test_set, n_shards, test_steps_per_shard,
    compression_type = "GZIP")

In [11]:
pprint.pprint(train_tfrecord_filenames)
pprint.pprint(valid_tfrecord_filenames)
pprint.pprint(test_tfrecord_fielnames)

['generate_tfrecords_zip/train_00000-of-00020',
 'generate_tfrecords_zip/train_00001-of-00020',
 'generate_tfrecords_zip/train_00002-of-00020',
 'generate_tfrecords_zip/train_00003-of-00020',
 'generate_tfrecords_zip/train_00004-of-00020',
 'generate_tfrecords_zip/train_00005-of-00020',
 'generate_tfrecords_zip/train_00006-of-00020',
 'generate_tfrecords_zip/train_00007-of-00020',
 'generate_tfrecords_zip/train_00008-of-00020',
 'generate_tfrecords_zip/train_00009-of-00020',
 'generate_tfrecords_zip/train_00010-of-00020',
 'generate_tfrecords_zip/train_00011-of-00020',
 'generate_tfrecords_zip/train_00012-of-00020',
 'generate_tfrecords_zip/train_00013-of-00020',
 'generate_tfrecords_zip/train_00014-of-00020',
 'generate_tfrecords_zip/train_00015-of-00020',
 'generate_tfrecords_zip/train_00016-of-00020',
 'generate_tfrecords_zip/train_00017-of-00020',
 'generate_tfrecords_zip/train_00018-of-00020',
 'generate_tfrecords_zip/train_00019-of-00020']
['generate_tfrecords_zip/valid_00000-of-

In [15]:
# 根据tfrecord文件读取数据
expected_features = {
    "input_features": tf.io.FixedLenFeature([8], dtype=tf.float32),
    "label": tf.io.FixedLenFeature([1], dtype=tf.float32)
}

def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example,
                                         expected_features)
    return example["input_features"], example["label"]

def tfrecords_reader_dataset(filenames, n_readers=5,
                             batch_size=32, n_parse_threads=5,
                             shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TFRecordDataset(
            filename, compression_type = "GZIP"),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_example,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

tfrecords_train = tfrecords_reader_dataset(train_tfrecord_filenames,
                                           batch_size = 3)
for x_batch, y_batch in tfrecords_train.take(2):
    print(x_batch)
    print(y_batch)

tf.Tensor(
[[ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]
 [-1.1157656   0.99306357 -0.334192   -0.06535219 -0.32893205  0.04343066
  -0.12785879  0.30707204]
 [-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[2.419]
 [0.524]
 [1.832]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]
 [-0.82195884  1.8741661   0.1821235  -0.03170019 -0.6011179  -0.14337493
   1.0852206  -0.8613995 ]
 [-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[1.832]
 [1.054]
 [1.832]], shape=(3, 1), dtype=float32)


In [16]:
batch_size = 32
tfrecords_train_set = tfrecords_reader_dataset(
    train_tfrecord_filenames, batch_size = batch_size)
tfrecords_valid_set = tfrecords_reader_dataset(
    valid_tfrecord_filenames, batch_size = batch_size)
tfrecords_test_set = tfrecords_reader_dataset(
    test_tfrecord_fielnames, batch_size = batch_size)

In [18]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu', input_shape=[8]),
    keras.layers.Dense(1),
])

model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2
)]

history = model.fit(tfrecords_train_set,
                   validation_data = tfrecords_valid_set,
                   steps_per_epoch = 11160 // batch_size,
                   validation_steps = 3870 // batch_size,
                   epochs = 100,
                   callbacks=callbacks)

Epoch 1/100
348/348 [==============================] - 1s 3ms/step - loss: 1.0849 - val_loss: 1.1384
Epoch 2/100
348/348 [==============================] - 1s 3ms/step - loss: 0.8767 - val_loss: 0.6993
Epoch 3/100
348/348 [==============================] - 1s 3ms/step - loss: 0.4490 - val_loss: 0.5167
Epoch 4/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3878 - val_loss: 0.4412
Epoch 5/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3663 - val_loss: 0.4266
Epoch 6/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3613 - val_loss: 0.4198
Epoch 7/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3477 - val_loss: 0.4073
Epoch 8/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3386 - val_loss: 0.4042
Epoch 9/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3371 - val_loss: 0.4006
Epoch 10/100
348/348 [==============================] - 1s 3ms/step - loss: 0.3418 - val_lo

In [19]:
model.evaluate(tfrecords_test_set, steps=5160 // batch_size)

161/161 [==============================] - 0s 2ms/step - loss: 0.3428


0.3428364098072052